![](https://raw.githubusercontent.com/MomsFriendlyRobotCompany/yivo/main/docs/yivo.png)

# Yivo

Kevin Walchko

---

This is a derivation off of MSP which seems to be `int`s rather than supporting `float`s for numbers. It also seems rather fragmented with different flavors of MSP. Due to the poor documentation, it is rather confusing.

I am going to use some of the ideas of MSP, but I won't really implement it.

| Header | Size | Type | Data        | Checksum |
|--------|------|------|-------------|----------|
| `$K`   | NN   | T    |`uint8_t[NN]`|`uint8_t` |

| Part | In Checksum | Type  |
|------|-------------|-------|
| Header |   | `uint8_t[2]`  |
| Size   | x | `uint16_t`    |
| Type   | x | `uint8_t`     |
| Data   | x | `uint8_t[NN]` |


**CHANGE**

| 0 | 1 | 2 | 3 | 4 | ... | -1 |
|---|---|---|---|---|-----|----|
|`$`|`K`| N | N | T | ... | checksum |

- Changes:
    - header change to `$K`
        - Header is changable, but has to be only 2 binary characters
    - packet size from 256 (8b) to 65535 (16b)
        - HN: high byte
        - LN: low byte
    - remove `>`, `<`, and `!` which didn't make a lot of sense honestly
        - An error message will be identified by a `MsgID`
- Keep:
    - XOR checksum
    - 256 (8b) different packet types
    - message ids can be used for request or response
    - all commands have response, either data from sensors or just an acknowledgement


# References

- github: [yivo](https://github.com/MomsFriendlyRobotCompany/yivo)
- [Python `struct` format](https://docs.python.org/3/library/struct.html#format-characters)

In [1]:
# reload library
%load_ext autoreload
%autoreload 2

In [78]:
from colorama import Fore
from yivo import Yivo, MsgIDs, Errors
from yivo import num_fields, checksum
from yivo.packet import chunk
import struct

In [97]:
def printMsg(msg):
    size, msgid, payload, cs = chunk(msg)
    print(f"[ {Fore.MAGENTA}",MsgIDs(msgid),f"{Fore.RESET} ]------------------------------")
    print(f" {Fore.CYAN}Size:{Fore.RESET} {size} bytes")
    print(f" {Fore.CYAN}Checksum:{Fore.RESET} {cs} / {hex(cs)}")
    print(f" {Fore.CYAN}Payload:{Fore.RESET} {payload}")

In [98]:
rr = Yivo(b'G')
msg = rr.pack(MsgIDs.RAW_IMU, range(9))
# print("TX:",msg)
printMsg(msg)

err,_,data = rr.unpack(msg)
print("RX:",data)

[  MsgIDs.RAW_IMU  ]------------------------------
 Size: 18 bytes
 Checksum: 124 / 0x7c
 Payload: b'\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00'
RX: IMU_RAW(ax=0, ay=1, az=2, gx=3, gy=4, gz=5, mx=6, my=7, mz=8)


In [49]:
msg = rr.pack(MsgIDs.IMU_AGMQT, range(15))
print("TX:",msg)

err,_,data = rr.unpack(msg)
print("RX:",data)

TX: b'GK<\x00\x8c\x00\x00\x00\x00\x00\x00\x80?\x00\x00\x00@\x00\x00@@\x00\x00\x80@\x00\x00\xa0@\x00\x00\xc0@\x00\x00\xe0@\x00\x00\x00A\x00\x00\x10A\x00\x00 A\x00\x000A\x00\x00@A\x00\x00PA\x0e\x00\x00\x00Q'
RX: ImuAGMQT(ax=0.0, ay=1.0, az=2.0, wx=3.0, wy=4.0, wz=5.0, mx=6.0, my=7.0, mz=8.0, qw=9.0, qx=10.0, qy=11.0, qz=12.0, temperature=13.0, ts=14)


In [50]:
msg = rr.pack(MsgIDs.IMU_AGMQT)
print("TX:",msg)

err,_,data = rr.unpack(msg)
print("RX:",data)

TX: b'GK\x00\x00\x8c\x8c'
RX: REQUEST(msgid=140)


In [6]:
for msgid in MsgIDs:
    print(msgid)

MsgIDs.REBOOT
MsgIDs.IDENT
MsgIDs.STATUS
MsgIDs.RAW_IMU
MsgIDs.SERVO
MsgIDs.MOTOR
MsgIDs.RC
MsgIDs.RAW_GPS
MsgIDs.COMP_GPS
MsgIDs.ANALOG
MsgIDs.RC_TUNING
MsgIDs.PID
MsgIDs.POSE
MsgIDs.WP
MsgIDs.SERVO_CONF
MsgIDs.IMU_AGMQT
MsgIDs.IMU_AGMT
MsgIDs.IMU_AGT
MsgIDs.IMU_AT
MsgIDs.MAGNETIC
MsgIDs.RANGE
MsgIDs.TEMP_PRES
MsgIDs.SET_PID
MsgIDs.SET_RC_TUNING
MsgIDs.ACC_CALIBRATION
MsgIDs.MAG_CALIBRATION
MsgIDs.GYR_CALIBRATION
MsgIDs.SET_WP
MsgIDs.SET_POSE
MsgIDs.SET_MOTOR
MsgIDs.YIVO_ERROR


In [103]:
def test_yivo():
    rr = Yivo()
    for k,v in list(Yivo.msgInfo.items()):
        # print(f"[ {Fore.GREEN}{k}{Fore.RESET} ]--------------------------------")
        fmt, obj = v
        # size = fmt.size - 6
        testsz = num_fields(obj)
        msg = rr.pack(k, range( testsz ))
        # print(f"  TX: {Fore.CYAN}{msg}{Fore.RESET}")
        printMsg(msg)

        err,_,data = rr.unpack(msg)
        if err > 0:
            print("Invalid msg:", err)
            break
        print(f"\n>> {data}\n")
        assert tuple(data) == tuple([float(x) for x in range(testsz)])
        
test_yivo()

[  MsgIDs.IDENT  ]------------------------------
 Size: 7 bytes
 Checksum: 99 / 0x63
 Payload: b'\x00\x01\x02\x03\x00\x00\x00'

>> IDENT(version=0, multitype=1, msp_version=2, capability=3)

[  MsgIDs.RAW_IMU  ]------------------------------
 Size: 18 bytes
 Checksum: 124 / 0x7c
 Payload: b'\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00'

>> IMU_RAW(ax=0, ay=1, az=2, gx=3, gy=4, gz=5, mx=6, my=7, mz=8)

[  MsgIDs.IMU_AGMQT  ]------------------------------
 Size: 60 bytes
 Checksum: 81 / 0x51
 Payload: b'\x00\x00\x00\x00\x00\x00\x80?\x00\x00\x00@\x00\x00@@\x00\x00\x80@\x00\x00\xa0@\x00\x00\xc0@\x00\x00\xe0@\x00\x00\x00A\x00\x00\x10A\x00\x00 A\x00\x000A\x00\x00@A\x00\x00PA\x0e\x00\x00\x00'

>> ImuAGMQT(ax=0.0, ay=1.0, az=2.0, wx=3.0, wy=4.0, wz=5.0, mx=6.0, my=7.0, mz=8.0, qw=9.0, qx=10.0, qy=11.0, qz=12.0, temperature=13.0, ts=14)

[  MsgIDs.IMU_AGMT  ]------------------------------
 Size: 44 bytes
 Checksum: 68 / 0x44
 Payload: b'\x00\x00\x00\x00\x00\x00\x80?\

In [54]:
from serial import Serial
import time

In [149]:
ser = Serial()
ser.port = "/dev/tty.usbmodem14601"
ser.baud = 9600
ser.timeout = 0.1
ser.open()
if not ser.is_open:
    print("serial not opened")

yivo = Yivo()
msg = yivo.pack(MsgIDs.IMU_AT)
print(msg)
ser.write(msg)
# ser.write(b"g")

retry = 5
while retry:
    if ser.in_waiting > 5:
        break
    time.sleep(0.1)
    retry -= 1
    
if retry > 0:
    msg = yivo.read_packet(ser)
    if msg is None:
        print("Crap!")
    else:
        print(">>",msg)
else:
    print("Crap ... nothing recv'd")
    
ser.close()

b'$K\x00\x00\x8f\x8f'
deque([b'$', b'K'], maxlen=2) b'\x14\x00' b'\x8f' b'\xcd\xcc\x8c?\xcd\xcc\x0c@33S@\xcd\xcc\x8c@Rd\x00\x00' b'\x8c'
>> ImuAT(ax=1.100000023841858, ay=2.200000047683716, az=3.299999952316284, temperature=4.400000095367432, ts=25682)


In [140]:
ser.close()
b''.join([b'\x14\x00', b'\x8f'])

b'\x14\x00\x8f'

In [127]:
# pkt = b'$K\x00\x14\x8f\xcd\xcc\x8c?\xcd\xcc\x0c@33S@\xcd\xcc\x8c'
pkt = b'$K\x14\x00\x8f\xcd\xcc\x8c?\xcd\xcc\x0c@33S@\xcd\xcc\x8c@\x01\x00\x00\x00\xbb'
yivo.unpack(pkt)

(<Errors.NONE: 0>,
 143,
 ImuAT(ax=1.100000023841858, ay=2.200000047683716, az=3.299999952316284, temperature=4.400000095367432, ts=1))

In [129]:
printMsg(pkt)

[  MsgIDs.IMU_AT  ]------------------------------
 Size: 20 bytes
 Checksum: 187 / 0xbb
 Payload: b'\xcd\xcc\x8c?\xcd\xcc\x0c@33S@\xcd\xcc\x8c@\x01\x00\x00\x00'


In [105]:
from collections import deque

r = deque(maxlen=2)
r.append(1)
print(r)
r.append(2)
print(r)
r.append(3)
print(r)

deque([1], maxlen=2)
deque([1, 2], maxlen=2)
deque([2, 3], maxlen=2)


In [136]:
b''.join([b'1',b'2'] + list(deque([3,4])))

TypeError: sequence item 2: expected a bytes-like object, int found